<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2025 European Union <br>
**License:** MIT <br>
**Authors:** Vinca Rosmorduc (CLS)

<html>
   <div style="float:left"><p>&emsp;</p></div>
   <div style="float:left"><a href="https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/applications/altimetry-applications"><img src="https://img.shields.io/badge/open-EUMETLAB-E67E22.svg"></a></div>
   <div style="float:left"><p>&emsp;</p></div>
   <div style="float:left"><a href="https://user.eumetsat.int/data/themes/marine"><img src="https://img.shields.io/badge/open-USER PORTAL-154360.svg"></a></div>
   <div style="float:left"><p>&emsp;</p></div>
   <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fapplications%2Faltimetry-applications/HEAD?labpath=2_methodological_approaches%2FAltimetry_corrections%2FAltimetry_corrections.ipynb"><img src="https://mybinder.org/badge_logo.svg"></a></div>
   <div style="float:left"><p>&emsp;</p></div>
   <div style="float:left"><a href="https://jupyterhub.prod.wekeo2.eu/hub/user-redirect/lab/tree/public/wekeo4oceans/altimetry-applications/2_methodological_approaches/Altimetry_corrections/Altimetry_corrections.ipynb"><img src="https://img.shields.io/badge/launch-WEKEO-1a4696.svg"></a></div>
   <div style="float:left"><p>&emsp;</p></div>
   <div style="float:left"><a href="https://code.insula.destine.eu/hub/"><img src="https://img.shields.io/badge/launch-DestinE-f43fd3.svg"></a></div></div>
</html>

# A look at L3 Marine Copernicus Service data vs L2 #

Datasets with a number of pre-processings and homogenizations done are available at the Marine Copernicus Service. Among them, the "L3" are most generally used for assimilation into ocean circulation models. 
What are their differences and uses with respect to the L2 Copernicus data available at EUMETSAT?

<br>made for EUMETSAT Copernicus Marine Data Service by V. Rosmorduc, CLS

### Import libraries

In [1]:
import shutil            # a library that allows us access to basic operating system commands like copy
import os                # a library that allows us access to basic operating system commands like making directories
import glob
import numpy as np
# reading / using NetCDF data
import xarray
import copernicusmarine
import eumdac
from pathlib import Path # a library that helps construct system path objects
import matplotlib
import matplotlib.pyplot as plt
from datetime import date, datetime
%matplotlib inline
import cartopy
import cartopy.crs as ccrs

Definition of download folders, time frame (only one day used here), colour bar extrema and the area

In [2]:
input_root = 'data/'
input_pathl3 = 'l3/'
input_path = 'l2_s6-3/'
output_l3 = input_root + input_pathl3
output_l2 = input_root + input_path

start = datetime(2025, 4, 8, 0, 0, 0)
end = datetime(2025, 4, 8, 23, 59, 59)
thedate = start

vmin=-0.25 ; vmax=0.25 ;
lon_min = -180 ; lon_max = 180 ; lat_min = -90 ; lat_max = 90 ;

Get the L3 data from Copernicus Marine Service for the given date (NB. this supposes that you have a .copernicusmarine-credentials file in a .copernicusmarine folder in your user directory.

In [ ]:
# complete current set of 1 Hz L3 files. Most also exists at 5 Hz in Near-Real Time (higher along-track resolution)
dataset_ids = [
	"cmems_obs-sl_glo_phy-ssh_nrt_al-l3-duacs_PT1S",
	"cmems_obs-sl_glo_phy-ssh_nrt_c2n-l3-duacs_PT1S",
	"cmems_obs-sl_glo_phy-ssh_nrt_h2b-l3-duacs_PT1S",
	"cmems_obs-sl_glo_phy-ssh_nrt_j3n-l3-duacs_PT1S",
	"cmems_obs-sl_glo_phy-ssh_nrt_s3a-l3-duacs_PT1S",
	"cmems_obs-sl_glo_phy-ssh_nrt_s3b-l3-duacs_PT1S",
	"cmems_obs-sl_glo_phy-ssh_nrt_s6a-hr-l3-duacs_PT1S",
	"cmems_obs-sl_glo_phy-ssh_nrt_swon-l3-duacs_PT1S",
]
#convert the date (string) in date format, extract year and month
datefile = thedate.strftime("%Y%m%d")
monthfolder = thedate.strftime("%m")
yearfolder = thedate.strftime("%Y")
datefilter = "*/" + yearfolder + "/" + monthfolder + "/"+ 'nrt_global_*_phy_l3_1hz_' + datefile + "_*.nc"

# Call the get function for each dataset to save files in output_directory, selected following the "filter" instruction
for dataset_id in dataset_ids:
    get_files = copernicusmarine.get(
        dataset_id=dataset_id,
        no_directories=True,
        output_directory=output_l3,
        filter=datefilter,
        )

INFO - 2025-04-30T13:56:21Z - Cache could not be loaded. Retrying without cache. If the problem with the cache persists, try running copernicusmarine describe --overwrite-metadata-cache                                                                                     | 0/3 [00:00<?, ?it/s]

ching catalog:   0%|                                                                                                                                                                                                                                                       | 0/3 [00:00<?, ?it/s]

Look at one of the downloaded files (Sentinel-6 Michael Freilich, aka Sentinel-6A or "s6a" file)

In [ ]:
input_file = 'nrt_global_s6a_hr_phy_l3_1hz_20250408_20250429.nc'
my_file = os.path.join(input_root,input_pathl3,input_file)
nc = xarray.open_dataset(my_file)
for variable in nc.variables.keys():
    print(variable)

You see there is a short list of variables in this dataset (with respect to the L2!). Let's plot one of those on a map.

In [ ]:
# map the data
latl3s6 = nc['latitude']
lonl3s6 = nc['longitude']
slal3s6 = nc['sla_filtered']

fig = plt.figure(figsize=(16,8))
ax = plt.axes(projection=ccrs.PlateCarree(), aspect=1.25)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.LAND, linewidth=1, facecolor='lightgrey', edgecolor='k', zorder=1)
# actually plot the data
tracks = ax.scatter(lonl3s6, latl3s6,  c=slal3s6, cmap='RdYlBu_r', vmin=vmin, vmax=vmax, marker='.', edgecolors='black', linewidth=0.01)
cbar = plt.colorbar(tracks)
plt.title('sea level anomaly filtered [m]', size=18)
plt.show()

You can see you have a full day worth of measurements, i.e. several tracks (or half-orbit) in the retrieved S6A file. Tracks and cycles are provided as variables, so that you can select them by number (see below). 

The other variables are either an alternate SLA, unfiltered (noisier but less smoothed). A few of the corrections applied in SLA computation are also provided, in particular tides. For example, if you have a more local tidal model and need to use it, you can thus remove the global model used to replace it by your own. You can also need to remove the ocean tides and the DAC if you want to see the effect of storm surges on a coast.

You also have mdt, to compute the absolute dynamic topography (adt = sla + mdt). 

The flag variable tracks whether you have NRT, STC or NTC data as inputs. 

Select one track, to compare it with the L2 equivalent.

In [ ]:
tracknumber=152
sat = 's6a'
#subset using the track number, and save the file.
subsetByTrack = nc.sel(time=(nc.track==tracknumber))
subsetByTrack.to_netcdf(output_l3 +sat+'track'+str(tracknumber)+'.nc')
nc.close()

fig = plt.figure(figsize=(16,8))
ax = plt.axes(projection=ccrs.PlateCarree(), aspect=1.25)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
ax.add_feature(cartopy.feature.LAND, linewidth=1, facecolor='lightgrey', edgecolor='k', zorder=1)
# actually plot the data
tracks = ax.scatter(subsetByTrack['longitude'], subsetByTrack['latitude'],  c=subsetByTrack['sla_filtered'], cmap='RdYlBu_r', vmin=vmin, vmax=vmax, \
                    marker='.', edgecolors='black', linewidth=0.01)
cbar = plt.colorbar(tracks)
plt.title('sea level anomaly [m]', size=18)
plt.show()

In [ ]:
# a simple plot over our subset, with light blue dots (b for blue, o for dot) for sla_unfiltered
# and a dark blue line for another variable, sla_filtered
plt.figure(figsize=(16,8))
plt.plot(subsetByTrack['latitude'], subsetByTrack['sla_unfiltered'], 'c.')
plt.plot(subsetByTrack['latitude'], subsetByTrack['sla_filtered'], c='blue')

On this plot, you notice the straight line corresponding to the part of the track crossing South America - you don't have any data measurement available there in this dataset. Around 50°S and 30-40°S, you are in turbulent areas, with strong eddies, shown here as succession of sharp increase/decrease half a meter in amplitude. You have eddies around 20°N.


You can test other track numbers - from 152 to 177. #176 shows the Gulf Stream turbulence when close to the North American continent.

Now, let's fetch the corresponding L2 data  from Sentinel-6 and 3 at EUMETSAT (which were used as inputs for the L3 for those missions).

For later use, we will download the whole period defined above, not only Sentinel-6 Michael Freilich track #152
(here also the credentials have been defined in a file in the user directory, this time as a "credentials" file in a .eumdac folder).

In [ ]:
# load credentials
file_credentials = True

if file_credentials == True:
    credentials_file = os.path.join(os.path.expanduser("~"),'.eumdac','credentials')
    credentials = Path(credentials_file).read_text().split(',')
    token = eumdac.AccessToken((credentials[0], credentials[1]))
    print(f"This token '{token}' expires {token.expiration}")
else:
    # see below for more information
    token = eumdac.AccessToken(("consumer_key", "consumer_secret"))

In [ ]:
datastore = eumdac.DataStore(token)

In [ ]:
collectionIDs = ["EO:EUM:DAT:0415", "EO:EUM:DAT:0855"]

In [ ]:
# set collection ID for L2 products
timeliness = 'ST'
for collectionID in collectionIDs:
    selected_collection = datastore.get_collection(collectionID)
    products = selected_collection.search(
        timeliness=timeliness,
        dtstart=start, dtend=end)
    for product in products:
        print(product)

Below, we will fetch only the Short Time Critical data, Reduced files for all missions. For Sentinel-3, we will rename the files using the dataset folder information.

In [ ]:
# set collection ID for L2 products
timeliness = 'ST'
for collectionID in collectionIDs:
    selected_collection = datastore.get_collection(collectionID)
    products = selected_collection.search(
        timeliness=timeliness,
        dtstart=start, dtend=end)

    for product in products:
        for entry in product.entries:
            if all(x in entry for x in ['S6A_P4_2__HR_RED__']):
                with product.open(entry=entry) as fsrc, open(os.path.join(os.getcwd(), output_l2, fsrc.name), mode='wb') as fdst:
                    shutil.copyfileobj(fsrc, fdst)
                    print(f'Download of file {fsrc.name} finished.')
            elif all(x in entry for x in ['reduced']):
                with product.open(entry=entry) as fsrc, open(os.path.join(os.getcwd(), output_l2, fsrc.name), mode='wb') as fdst:
                    shutil.copyfileobj(fsrc, fdst)
                    shutil.copy(output_l2 + fsrc.name, output_l2 + entry[:-28]+'.nc')
                    print(f'Copy of file {entry[:-28]+".nc"} finished.')

We begin by opening Sentinel-6 track #152, and only that one.

In [ ]:

input_file = 'S6A_P4_2__HR_RED__ST_162_152_20250407T234227_20250408T003840_G01.nc'
my_file = os.path.join(output_l2,input_file)
nc01 = xarray.open_dataset(my_file, group='data_01')
nc01ku = xarray.open_dataset(my_file, group='data_01/ku')

# for xarray version >= 2024.10, you can use
#nc01 = xarray.open_datatree(my_file)["/data_01"].dataset
#nc01ku = xarray.open_datatree(my_file)["/data_01/ku"].dataset
# (more information are kept since inherited in the datatree structure, but you still need 2 different datasets):

latl2 = nc01['latitude']
lonl2 = nc01['longitude']
timel2 = nc01['time']
sshal2 = nc01ku['ssha']

# flag to use to remove spurious and non ocean data ;
flags_ssha = nc01.surface_classification_flag + nc01ku.range_ocean_qual + nc01ku.sig0_ocean_qual + nc01.rain_flag + nc01ku.swh_ocean_qual \
        + nc01.meteo_map_availability_flag + nc01.mean_dynamic_topography_qual + nc01.mean_sea_surface_sol1_qual + nc01.rad_sea_ice_flag \
        + nc01.rad_wet_tropo_cor_qual

# Use of the flag is commented here so that you may see those spurious and non ocean data. To "edit" the data, just uncomment the following line

#sshal2[flags_ssha != 0.0] = np.nan

fig = plt.figure(figsize=(16,8))
ax = plt.axes(projection=ccrs.PlateCarree(), aspect=1.25)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.LAND, linewidth=1, facecolor='lightgrey', edgecolor='k', zorder=1)
ax.add_feature(cartopy.feature.RIVERS, linewidth=1, edgecolor='blue', zorder=3)
ax.add_feature(cartopy.feature.LAKES, linewidth=1, facecolor='blue', edgecolor=None, zorder=2)
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
# actually plot the data
tracks = ax.scatter(lonl2, latl2, c=sshal2, cmap='RdYlBu_r', vmin=vmin, vmax=vmax, marker='.', edgecolors='black', linewidth=0.01, zorder=4)
cbar = plt.colorbar(tracks)
plt.title('SSHA S6 [m]', size=18, pad=24)
plt.savefig("ssha_s6_tr050.png", bbox_inches="tight")
plt.show()



In [ ]:
nc01ku

Compared with the L3 equivalent, you may notice a grey part on the track on the continents: for S6, the L2 data files include data over continents, which can be used - mostly over continental waters. Some other continental applications have been studied (deserts, snow), and of course, even if S6 is not going very high north and south, you can have data over sea ice and continental ice. 
SSHA is not computed outside ice-free ocean, which leads to the grey parts of the track (grey meaning that you do have data coordinates for those points but that the variable value there is "Not a Number"), but range, altitude, and some of the corrections can be available and exploited over other surfaces. Others aren't - the most obvious being ocean tide, but Sea State Bias is not, nor radiometer wet tropospheric correction; or MSS/MDT (geoid is). 
On the map, you can also see that some L2 data are over the continent. We look at the land-mask with rivers and lakes we also plotted, they are over the Great Lakes (over some large lakes, ocean processing and corrections can be used; over smaller ones, different algorithms and corrections must be used). 

In [ ]:
# we do a simple plot over our subset, with blue dots (b for blue, o for dot) for sla_filtered
# and a red line for another variable, sla_unfiltered
plt.figure(figsize=(16,8))
#plt.plot(subsetByTrack['latitude'], subsetByTrack['sla_filtered'], 'bo')
plt.plot(subsetByTrack['latitude'], subsetByTrack['sla_unfiltered'], 'bo')
plt.plot(latl2, sshal2, c='red')

In [ ]:
# we do a simple plot over our subset, with blue dots (b for blue, o for dot) for sla_filtered
# and a red line for another variable, sla_unfiltered
plt.figure(figsize=(16,8))
#plt.plot(subsetByTrack['latitude'], subsetByTrack['sla_filtered'], 'bo')
plt.plot(subsetByTrack['latitude'], subsetByTrack['sla_unfiltered'], 'bo')
plt.plot(latl2, sshal2, c='red')
plt.ylim([-1, 1])

In [ ]:
# we do a simple plot over our subset, with light blue dots (c for cyan blue, . for dot) for sla_filtered
# and a dark blue line for another variable, sla_unfiltered
plt.figure(figsize=(16,8))
plt.plot(subsetByTrack['latitude'], subsetByTrack['sla_unfiltered'], 'c.')
plt.plot(subsetByTrack['latitude'], subsetByTrack['sla_filtered'], c='blue')
plt.plot(latl2, sshal2, c='red')
plt.axis([10, 45, -1, 0.75])

Differences:
- values are removed in L3 because potentially spurious (aka "editing"), e.g. when outliers see ~16-17.5°N, track 152 S6a, or when there is high varibility in a short segment. The main editing for L3 is done using physical thresholds on some of the measurements. However, sometimes high values or high variability can be real - and interesting to study. So, if you're interested in extreme events, the L2 could be more useful for you.
  NB. Flags are provided in L2 (not applied here for the sake of demonstration) to remove spurious data, and also to mask lands to keep only ocean or only sea ice, etc.
- The L3 data begin in fact quite far from the North American continent (the continent is at about 36°N - NB. the track is "descending", from North to South), in fact due to the file granularity definition - this particular track begins on April 7... and only the part dated from April 8 is in the L3 here. 
- "A long-wavelength error" is applied to L3 for homogenization between missions. In Sentinel-6 case its amplitude is not very high, though, since it is the reference for all contemporary missions.
- Corrections and reference surfaces are homogeneous in L3 for all missions as far as possible, and up to the latest recommended versions at the point of reprocessing (for the REP data) or at the time of processing (for the NRT data), so some can be different from the ones used in L2. Due to the very recent upgrade in the L2 ground segment (as of April 7 ,2025), both L2 and L3 sets of corrections are currently in line (late April - May 2025!)

S6 track WITH editing

In [ ]:
sshal2[flags_ssha != 0.0] = np.nan
# we do a simple plot over our subset, with light blue dots (c for cyan blue, . for dot) for sla_filtered
# and a dark blue line for another variable, sla_unfiltered
plt.figure(figsize=(16,8))
plt.plot(subsetByTrack['latitude'], subsetByTrack['sla_unfiltered'], 'c.')
plt.plot(subsetByTrack['latitude'], subsetByTrack['sla_filtered'], c='blue')
plt.plot(latl2, sshal2, c='red')
plt.axis([10, 45, -1, 0.75])

Most spurious data (outliers), as well as the continental water have been removed. 

# Using several missions together

Map all L3 tracks (all sat) for a given day

In [ ]:
sat= '*'
l3files = glob.glob(input_root + input_pathl3 + 'nrt_global_' + sat + '_phy_l3_1hz_*.nc')
l3files.sort()

fig = plt.figure(figsize=(16,8))
ax = plt.axes(projection=ccrs.PlateCarree(), aspect=1.25)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.LAND, linewidth=1, facecolor='lightgrey', edgecolor='k', zorder=1)
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')

# here we open and plot each file the one after the other
for atfile in l3files:
    # open file and select data
    ncat = xarray.open_dataset(atfile)
    lat = ncat['latitude']
    lon = ncat['longitude']
    vartracks = ncat['sla_filtered']
    #vartracks = ncat['sla'] +  ncat['dac'] + ncat['ocean_tide'] + ncat['internal_tide'] + ncat['lwe']
    tracks = ax.scatter(lon, lat, c=vartracks, cmap='RdYlBu_r', vmin=vmin, vmax=vmax, marker='.', edgecolors='grey', linewidth=0.1, zorder=3)

cbar = plt.colorbar(tracks)
plt.title('SSHA Level 3 - all missions, one day [m]', size=18, pad=24)
#plt.savefig("ssha_s6_tr050.png", bbox_inches="tight")
plt.show()


On this map, you see the daily tracks of 7 different altimetry missions (Saral, Cryosat-2, HY-2B, Sentinel-3A & B, Sentinel-6 Michael Freilich and Swot nadir). On another day you may have had a different count - it depends mostly on launches and calibration phases, and on ends of mission but sometimes a satellite can also have a temporary unavailability - and definitely a different pattern. Since 2018, though, there was always at least 5 altimetry satellites measuring over different tracks (at some point Jason-3 and Sentinel-6 Michael Freilich were both in flight and measuring, but the one following the other closely for intercalibration; only one of the two is kept for L3, in this case), and often more than 6.

NB. The map seems quite well filled in, but note that the line width is not plotted with respect to the real footprint of the altimeters.
It is enough to glimpse a number of patterns, though - the high variability in the major currents, a high/low large pattern in the Equatorial Pacific (an El Niño if any). L4 grids are made from this, and from past (and future if possible) days.


Map all L2 tracks (S3A&B+S6) for the same day. We will open three different combined datasets, one for each satellite. 

In [ ]:
# here we can open the files for each mission within the same dataset(s), the dimension (time or time_01) being continuous from one file to the 
# following one
# Sorting the files is important here, to have a time increasing properly over the whole opened datasets. 
# NB. we could have plotted one file after the other, too, though - but to use them, having them all in the same xarray dataset is easier

# Sentinel-6 data are stored in groups; we have to open the different groups in different datasets
s6_files = glob.glob(os.path.join(output_l2,'S6A_P4_2__HR_RED__ST_*.nc'))
s6_files.sort()
s6data01 = xarray.open_mfdataset(s6_files, group='data_01', combine='nested', concat_dim='time', compat='no_conflicts')
s6data01ku = xarray.open_mfdataset(s6_files, group='data_01/ku', combine='nested', concat_dim='time', compat='no_conflicts')
lats6l2 = s6data01['latitude']
lons6l2 = s6data01['longitude']
sshas6l2 = s6data01ku['ssha']

s3a_files = glob.glob(os.path.join(output_l2,'S3A_SR_2_WAT____*.nc'))
s3a_files.sort()
s3adata = xarray.open_mfdataset(s3a_files, combine='nested', concat_dim='time_01', compat='no_conflicts')
lats3a = s3adata['lat_01']
lons3a = s3adata['lon_01']
sshas3a = s3adata['ssha_01_ku']

s3b_files = glob.glob(os.path.join(output_l2,'S3B_SR_2_WAT____*.nc'))
s3b_files.sort()
s3bdata = xarray.open_mfdataset(s3b_files, combine='nested', concat_dim='time_01', compat='no_conflicts')
lats3b = s3bdata['lat_01']
lons3b = s3bdata['lon_01']
sshas3b = s3bdata['ssha_01_ku']

fig = plt.figure(figsize=(16,8))
ax = plt.axes(projection=ccrs.PlateCarree(), aspect=1.25)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.LAND, linewidth=1, facecolor='lightgrey', edgecolor='k', zorder=1)
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
# actually plot the data
trackss6 = ax.scatter(lons6l2, lats6l2, c=sshas6l2, cmap='RdYlBu_r', vmin=vmin, vmax=vmax, marker='.', edgecolors='black', linewidth=0.01)
trackss3a = ax.scatter(lons3a, lats3a, c=sshas3a, cmap='RdYlBu_r', vmin=vmin, vmax=vmax, marker='.', edgecolors='black', linewidth=0.01)
trackss3b = ax.scatter(lons3b, lats3b, c=sshas3b, cmap='RdYlBu_r', vmin=vmin, vmax=vmax, marker='.', edgecolors='black', linewidth=0.01)

cbar = plt.colorbar(trackss6)
plt.title('SSHA [m]', size=18, pad=24)
plt.savefig("ssha_s6s3ab_day.png", bbox_inches="tight")
plt.show()


(remember that we plotted "only" 3 satellites, while for the L3 map you had 7 of them)

# Take-home messages

## Uses of L2 which can't be done with L3

- With L2, you can change "retracking" (the way the range, sigma0, wind, SWH,...  are computed from the radar signal waveform) among the ones proposed. Some may fit better your needs than the one chosen in the L3!
- You can also access e.g., the ionospheric correction or another correction not included in the L3, and replace it by another, provided one, or one of your own, or study it for its own, etc.
- L2 are the solution when you're working out of "normal conditions over open ocean" -- which can be just working close to the coasts, or on extreme events, over sea ice; or even land waters or any land-reflecting surface. 
- L2 can also be used to check on the origin of a gap in L3. They are organized by tracks (parts for NRT, full tracks in STC and NTC), even when over two different days.
- you have both Significant wave heights and Sea surface height anomalies in the same files (two different data products for the L3)

- With the "enhanced" L2 (Sentinel-3; L1B for Sentinel-6) you can work on the waveform.

## When L3 should be preferred

Basically, all the L3 possible uses are also possible with the L2. However:
- All the data from the different missions are homogenized in the L3 (cross-calibration and also in the applied corrections and algorithms as much as possible). This is important if you plan on working with several different missions. Otherwise, you may have biases and some discrepancies between two different missions. This is why they are mostly preferred to L2 for assimilation into ocean models. 

- Moreover Marine Copernicus Service will provide you with a "one-stop-shop" for the L3 from all available missions - while, to fetch all the L2 used above, you'd need to connect to at least 4 different data centers (Space agencies).

- L3 have less variables, so are easier to use in this respect, and the same physical variable is always called the same name in all L3 files, which is not the case for L2 (and can complicate reading codes)